In [8]:
import os
from bs4 import BeautifulSoup
import nltk
#nltk.download('punkt')
from nltk.corpus import stopwords
from keybert import KeyBERT
import json
import tqdm
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer

In [9]:

# Ruta de la carpeta que contiene los archivos origen
folder_path = "../data/splits/train/"  

# Lista creada para almacenar el texto del cuerpo de todas las páginas
all_page_texts = []

# Function defined to extract the title and summary from the page.
def extract_title_and_summary(file_path,dirs, limit_link=3,sentences_count=1,long_text=1024):
    with open(file_path, 'r', encoding='latin-1') as file:
        text = file.read()

    # Analyze the HTML content using BeautifulSoup.
    soup = BeautifulSoup(text, 'html.parser')

    # Extract the page title
    title = soup.title.string if soup.title else "No title"

    links = []
    for a in soup.find_all('a'):
        if 'href' in a.attrs:
            if len(links) < limit_link:
                links.append(a['href'])
            else:
                break
        #print("Enlaces en la página:", links)

    # Extract the text from the page's body
    page_text = ' '.join(soup.stripped_strings)

    # Perform summarization using sumy
    parser = PlaintextParser.from_string(page_text, Tokenizer("english"))
    summarizer = LexRankSummarizer()
    summary = summarizer(parser.document, sentences_count=sentences_count)  # You can adjust the number of sentences in the summary.
    #print(summary)
    summary_text = ' '.join([str(sentence) for sentence in summary])

    return {"Title": title, "Body": summary_text, "link": links ,"ground_truth": dirs,"1024_text":text[:long_text]}


In [10]:
# Folder path containing the original files
#folder_path = "C:/Users/user/Downloads/dataset_splitted/splits/test"
folder_path = "../data/splits/train"

# Output folder path for JSON files
#output_folder = "C:/Users/user/Downloads/dataset_splitted/splits/test_json"
output_folder = "../data/splits/train_1024/"

os.makedirs(output_folder, exist_ok=True)
# Process all files in the folder
for dirs in os.listdir(folder_path):
    for filename in tqdm.tqdm(os.listdir(os.path.join(folder_path, dirs)), desc="Extrayendo datos"):
        file_path = os.path.join(folder_path, dirs, filename)
        #print(file_path)
        if os.path.isfile(file_path):
            formatted_data = extract_title_and_summary(file_path,dirs)
            #print(formatted_data)
            #print("Processed:", filename)
            #print("Title:", formatted_data["Title"])
            #print("Summary:", formatted_data["Body"])
            #print("Ground Truth:", formatted_data["ground_truth"])
            # Save the data to a JSON fileç
            output_folder_dir=os.path.join(output_folder,dirs)
            os.makedirs(output_folder_dir, exist_ok=True)
            output_json_file = os.path.join(output_folder_dir, os.path.splitext(filename)[0] + ".json")
            with open(output_json_file, 'w', encoding='utf-8') as json_file:
                json.dump(formatted_data, json_file, ensure_ascii=False, indent=4)

Extrayendo datos:  10%|██▏                   | 302/3011 [00:10<01:04, 42.20it/s]<ipython-input-9-b4d5ecffc733>:13: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')
Extrayendo datos: 100%|███████████████████████| 744/744 [00:09<00:00, 81.02it/s]
